<a href="https://colab.research.google.com/github/Spencer166/Capstone/blob/main/Step_11%20-%20Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 11: Deployment

**Project Overview:** This project builds and deploys a Credit Risk Prediction system that estimates the probability that a loan applicant will repay a loan. A machine learning pipeline is trained using historical loan data and deployed as a Flask web application that allows users to input applicant information and receive real-time predictions. I have split this notebook into two different sections, Python and Docker:

**Python:** The Python section is code that should be run as is within this notebook. Running this code will produce two important files, gradient_boosting_pipeline.joblib and app.py. Once these files have been created, download them and save them to a folder on your desktop called "Docker".

**Docker:** The Docker section is code that will not be run in this notebook but will be used to containerize the application. I included the code in this notebook for documentation purposes. See the Docker section in this notebook for futher instructions.

# Python

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
import joblib

In [2]:
# Path to the dataset
file_path = "/content/drive/MyDrive/train.csv"

# Load into DataFrame
df_full = pd.read_csv(file_path)

# Sample 100,000 rows
df = df_full.sample(n=100000, random_state=1)

# Drop id column
df = df.drop('id', axis=1)

In [3]:
# Define Target
target = "loan_paid_back"

# Split into features and Target
X = df.drop(columns=[target])
y = df[target]

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

# Separate numeric and categorical columns
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns


# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(drop="first",handle_unknown='ignore'), cat_cols)])

In [4]:
# Gradient Boosting model with fixed hyperparameters
gb_model = GradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=4,
    n_estimators=200,
    random_state=42)


# Create pipeline with Gradient Boosting Classifier
model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", gb_model)])

# Fit to the training data
model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount',
       'interest_rate'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['gender', 'marital_status', 'education_level', 'employment_status',
       'loan_purpose', 'grade_subgrade'],
      dtype='object'))])),
                ('model',
                 GradientBoostingClassifier(max_depth=4, n_estimators=200,
                                            random_state=42))])

In [5]:
# Save Trained Model
joblib.dump(model, "gradient_boosting_pipeline.joblib")

['gradient_boosting_pipeline.joblib']

In [6]:
!pip install flask pyngrok joblib

In [7]:
%%writefile app.py

from flask import Flask, request, render_template_string
import joblib
import pandas as pd
import traceback

app = Flask(__name__)

# Load Trained Model
model = joblib.load("gradient_boosting_pipeline.joblib")

# Create HTML Template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Credit Risk Prediction</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f6f8;
        }
        .container {
            width: 480px;
            margin: 40px auto;
            padding: 20px;
            background: white;
            border-radius: 8px;
            box-shadow: 0px 0px 12px rgba(0,0,0,0.1);
        }
        h2 {
            text-align: center;
        }
        label {
            margin-top: 10px;
            display: block;
            font-weight: bold;
        }
        input, select, button {
            width: 100%;
            padding: 8px;
            margin-top: 5px;
        }
        button {
            margin-top: 15px;
            background-color: #007BFF;
            color: white;
            border: none;
            cursor: pointer;
        }
        .result {
            margin-top: 20px;
            padding: 10px;
            background-color: #eef;
            border-radius: 5px;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Credit Risk Prediction</h2>

        <form method="POST" action="/predict">

            <label>Annual Income</label>
            <input type="number" name="annual_income" required>

            <label>Debt-to-Income Ratio</label>
            <input type="number" step="0.01" name="debt_to_income_ratio" required>

            <label>Credit Score</label>
            <input type="number" name="credit_score" required>

            <label>Loan Amount</label>
            <input type="number" name="loan_amount" required>

            <label>Interest Rate</label>
            <input type="number" step="0.01" name="interest_rate" required>

            <label>Gender</label>
            <select name="gender" required>
                <option value="">Select</option>
                <option value="Male">Male</option>
                <option value="Female">Female</option>
                <option value="Other">Other</option>
            </select>

            <label>Marital Status</label>
            <select name="marital_status" required>
                <option value="">Select</option>
                <option value="Single">Single</option>
                <option value="Married">Married</option>
                <option value="Divorced">Divorced</option>
            </select>

            <label>Education Level</label>
            <select name="education_level" required>
                <option value="">Select</option>
                <option value="High School">High School</option>
                <option value="Bachelor">Bachelor</option>
                <option value="Master">Master</option>
                <option value="PhD">PhD</option>
            </select>

            <label>Employment Status</label>
            <select name="employment_status" required>
                <option value="">Select</option>
                <option value="Employed">Employed</option>
                <option value="Self-Employed">Self-Employed</option>
                <option value="Unemployed">Unemployed</option>
            </select>

            <label>Loan Purpose</label>
            <select name="loan_purpose" required>
                <option value="">Select</option>
                <option value="Debt Consolidation">Debt Consolidation</option>
                <option value="Home Improvement">Home Improvement</option>
                <option value="Education">Education</option>
                <option value="Medical">Medical</option>
                <option value="Other">Other</option>
            </select>

            <label>Grade / Subgrade</label>
            <input type="text" name="grade_subgrade" placeholder="e.g., B3" required>

            <button type="submit">Predict Credit Risk</button>
        </form>

        {% if prediction is not none %}
        <div class="result">
            <p>Prediction: {{ prediction }}</p>
            <p>Payback Probability: {{ probability }}</p>
        </div>
        {% endif %}

    </div>
</body>
</html>
"""

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE, prediction=None)

@app.route("/predict", methods=["POST"])
def predict():
    try:

        # Read form inputs
        input_data = {
            "annual_income": float(request.form["annual_income"]),
            "debt_to_income_ratio": float(request.form["debt_to_income_ratio"]),
            "credit_score": float(request.form["credit_score"]),
            "loan_amount": float(request.form["loan_amount"]),
            "interest_rate": float(request.form["interest_rate"]),
            "gender": request.form["gender"],
            "marital_status": request.form["marital_status"],
            "education_level": request.form["education_level"],
            "employment_status": request.form["employment_status"],
            "loan_purpose": request.form["loan_purpose"],
            "grade_subgrade": request.form["grade_subgrade"]
        }


        # Convert to DataFrame
        input_df = pd.DataFrame([input_data])


        # Model inference
        prediction = model.predict(input_df)[0]
        probability = model.predict_proba(input_df)[0][1]

        result_text = (
            "Repay"
            if prediction == 1
            else "Default"
        )


        # Return result
        return render_template_string(
            HTML_TEMPLATE,
            prediction=result_text,
            probability=round(float(probability), 3)
        )

    except Exception:

        # Detailed error output
        return (
            f"<h3>Prediction Error</h3>"
            f"<pre>{traceback.format_exc()}</pre>",
            500
        )
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)

Writing app.py


# Docker

**Dockerfile:** Copy and paste the below code to a notepad file and name it "Dockerfile". Note, do not use the .txt extention. There should not be any type selected. Save it to the same Docker folder mentioned previously.

In [ ]:
# Base image
FROM python:3.12-slim

# Set working directory
WORKDIR /app

# Copy dependency file
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy app code and model
COPY app.py .
COPY gradient_boosting_pipeline.joblib .

# Expose Flask port
EXPOSE 5000

# Start Flask app
CMD ["python", "app.py"]

**Requirements:** Copy and paste the below code to a notepad file and name it "requirements". Save it to the Docker folder and make sure to save it with the .txt extention.

In [ ]:
flask
pandas
numpy
scikit-learn==1.6.1
joblib

**Containerize:** To containerize the application, open terminal within the Docker folder on your desktop. Then run the below commands:

In [ ]:
#Build the Docker Image
docker build -t credit-risk-app .

#Run the Docker Container
docker run -p 5000:5000 credit-risk-app